In [ ]:
from __future__ import absolute_import, division, print_function

import argparse
import importlib
import itertools
import matplotlib
matplotlib.use('Agg')
import time
from   multiprocessing import Pool
import numpy as np
import os
import pdb
import pickle
import subprocess
import sys
import tensorflow as tf
# import tensorflow.compat.v1 as tf
import tf_slim as slim
import threading
import scipy.misc
from skimage import color
from taskonomy.taskbank.lib.models.sample_models import *
from taskonomy.taskbank.lib.data.synset import *
import scipy
import skimage
import skimage.io
import taskonomy.code.tools.init_paths
import transforms3d
import math
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from taskonomy.taskbank.tools.task_viz import *
import random
import taskonomy.taskbank.tools.utils as utils
import models.architectures as architectures
from   data.load_ops import resize_rescale_image
from   data.load_ops import rescale_image
import lib.data.load_ops as load_ops

os.environ["CUDA_VISIBLE_DEVICES"]="0"


In [ ]:

tf.compat.v1.disable_eager_execution()


img = '/mnt/raid/data/agnessa/val_256/Places365_val_00015550.jpg/'
task = 'edge2d'
store = 'mnt/raid/ni/agnessa/RSA/Scenes/Places365' 

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

def generate_cfg(task):
    
    repo_dir = '/mnt/raid/ni/agnessa/RSA/Edge_detection/taskonomy/taskbank/'
    CONFIG_DIR = os.path.join(repo_dir, 'experiments/final', task)
    
    ############## Load Configs ##############
    import taskonomy.taskbank.tools.utils as utils
    import data.load_ops as load_ops
    from   general_utils import RuntimeDeterminedEnviromentVars
    cfg = utils.load_config( CONFIG_DIR, nopause=True )
    RuntimeDeterminedEnviromentVars.register_dict( cfg )
    cfg['batch_size'] = 1
    if 'batch_size' in cfg['encoder_kwargs']:
        cfg['encoder_kwargs']['batch_size'] = 1
    cfg['model_path'] = os.path.join( repo_dir, 'temp', task, 'model.permanent-ckpt' )
    cfg['root_dir'] = repo_dir
    return cfg

def run_to_task():
    import general_utils
    from   general_utils import RuntimeDeterminedEnviromentVars

    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

    
    img = '/mnt/raid/data/agnessa/val_256/Places365_val_00015550.jpg/'
    task = 'edge2d'
    store = 'mnt/raid/ni/agnessa/RSA/Scenes/Places365' 

    img = load_raw_image_center_crop( img )
    img = Image.fromarray(np.squeeze(img))
    cfg = generate_cfg(task)

    # Since we observe that areas with pixel values closes to either 0 or 1 sometimes overflows, we clip pixels value
    cfg['input_preprocessing_fn'] = load_ops.resize_rescale_image_low_sat
    img = cfg[ 'input_preprocessing_fn' ]( img, **cfg['input_preprocessing_fn_kwargs'] )
    img = img[np.newaxis,:]
    print("Doing {task}".format(task=task))
    general_utils = importlib.reload(general_utils)
    
    from tensorflow.python.framework import ops
    ops.reset_default_graph()

    training_runners = { 'sess': tf.compat.v1.InteractiveSession(), 'coord': tf.train.Coordinator() }

    ############## Set Up Inputs ##############
    # tf.logging.set_verbosity( tf.logging.INFO )
    setup_input_fn = utils.setup_input
    inputs = setup_input_fn( cfg, is_training=False, use_filename_queue=False )
    RuntimeDeterminedEnviromentVars.load_dynamic_variables( inputs, cfg )
    RuntimeDeterminedEnviromentVars.populate_registered_variables()
    start_time = time.time()

    ############## Set Up Model ##############
    model = utils.setup_model( inputs, cfg, is_training=False )
    m = model[ 'model' ]
    model[ 'saver_op' ].restore( training_runners[ 'sess' ], cfg[ 'model_path' ] )
    
    init = tf.compat.v1.global_variables_initializer()

    with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True)) as sess:
        sess.run(init)
        sess.graph.get_operations()
        layer = sess.graph.get_tensor_by_name('encoder/block2/unit_1/bottleneck_v1/conv1/Conv2D:0')
        print('got layer')
    
        representation = training_runners['sess'].run( 
            [ layer ], feed_dict={m.input_images: img} )
        print('got representation')
    

#         s_name, file_extension = os.path.splitext(args.store_name)
#         with open('{}.npy'.format(s_name), 'wb') as fp:
#             np.save(fp, np.squeeze(representation))



#         just_rescale = ['autoencoder', 'denoise', 'edge2d', 
#                     'edge3d', 'keypoint2d', 'keypoint3d',
#                     'reshade', 'rgb2sfnorm' ]

#         if task in just_rescale:
#             simple_rescale_img(predicted, args.store_name)
#             return
    

    ############## Clean Up ##############
    training_runners[ 'coord' ].request_stop()
    training_runners[ 'coord' ].join()
    print("Done: {}".format(config_name))

    ############## Reset graph and paths ##############            
    tf.reset_default_graph()
    training_runners['sess'].close()
    return

if __name__ == '__main__':
    run_to_task()

